# **Downloading the dataset using kaggle API**

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
 !mkdir ~/.kaggle

In [ ]:
# copy the json file to the folder .kaggle in the root directory
!cp kaggle.json ~/.kaggle/

In [ ]:
# change folder permissions to be able to read and write
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pwd

In [ ]:
!mkdir dataset

In [ ]:
%cd /content/dataset

In [ ]:
!pwd

In [ ]:
# https://www.kaggle.com/alessiocorrado99/animals10
!kaggle datasets download -d alessiocorrado99/animals10

# **Unzipping the dataset**

In [ ]:
!unzip -x /content/dataset/animals10.zip

In [ ]:
%cd /content/dataset/raw-img

In [ ]:
import os

In [ ]:
print(f"We have a total number of {len(os.listdir('./'))} classes")

In [ ]:
for folder in os.listdir('./'):
  images = [x for x in os.listdir(f'./{folder}') if (x.endswith('.jpg') or x.endswith('.jpeg'))]
  print(f'{folder} : {len(images)} images')

# **Reading and showing the image**

In [ ]:
import cv2 as cv
from google.colab.patches import cv2_imshow

In [ ]:
img = cv.imread('/content/dataset/raw-img/pecora/OIP-1E6UPBzw9JX-ysAKA8VuUQHaFj.jpeg')

In [ ]:
cv2_imshow(img)

In [ ]:
img.shape

In [ ]:
 img_resized = cv.resize(img, (128, 64))

In [ ]:
img_resized.shape

In [ ]:
cv2_imshow(img_resized)

In [ ]:
img = cv.imread('/content/dataset/raw-img/pecora/OIP-1E6UPBzw9JX-ysAKA8VuUQHaFj.jpeg', 0)

In [ ]:
cv2_imshow(img)

In [ ]:
img.shape

# **Data generator**

In [ ]:
import tensorflow.keras as keras
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
CLASSES_labels = {}
for index, folder in enumerate(os.listdir('./')):
  CLASSES_labels[folder] = index

In [ ]:
CLASSES_labels

In [ ]:
images_paths = []
images_labels = []
for folder in os.listdir('./'):
  images = [folder + '/' + x for x in os.listdir(f'./{folder}') if (x.endswith('.jpg') or x.endswith('.jpeg'))]
  images_paths = images_paths + images
  label = CLASSES_labels[folder]
  images_labels = images_labels + [label] * len(images)

In [ ]:
len(images_paths), len(images_labels)

In [ ]:
print(list(zip(images_paths[::1000], images_labels[::1000])))

In [ ]:
# train test split
images_paths_train, images_paths_test, images_labels_train, images_labels_test = train_test_split(images_paths, images_labels, test_size=0.1, stratify=images_labels, random_state=17)
images_paths_train, images_paths_validation, images_labels_train, images_labels_validation = train_test_split(images_paths_train, images_labels_train, test_size=0.1, stratify=images_labels_train, random_state=17)

In [ ]:
len(images_paths_train), len(images_paths_validation), len(images_paths_test)

In [ ]:
# data generator
class DataGenerator(keras.utils.Sequence):
  'Generates data for Keras'
  def __init__(self, list_IDs, labels, batch_size=16, dim=(64,128), n_channels=3,
              n_classes=10, shuffle=True):
    'Initialization'
    self.dim = dim
    self.batch_size = batch_size
    self.labels = labels
    self.list_IDs = list_IDs
    self.n_channels = n_channels
    self.n_classes = n_classes
    self.shuffle = shuffle
    self.on_epoch_end()

  def __len__(self):
    'Denotes the number of steps per epoch'
    return int(np.floor(len(self.list_IDs) / self.batch_size))

  def __getitem__(self, index):
    'Generate one batch of data'
    # Generate indexes of the batch
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Find list of IDs
    list_IDs_temp = [self.list_IDs[k] for k in indexes]
    list_labels_temp = [self.labels[k] for k in indexes]

    # Generate data
    X, y = self.__data_generation(list_IDs_temp, list_labels_temp)

    return X, y

  def on_epoch_end(self):
    'Updates indexes after each epoch'
    self.indexes = np.arange(len(self.list_IDs))
    if self.shuffle == True:
        np.random.shuffle(self.indexes)

  def __data_generation(self, list_IDs_temp, list_labels_temp):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    X = np.empty((self.batch_size, *self.dim, self.n_channels))
    y = np.empty((self.batch_size), dtype=int)

    # Generate data
    for i, ID in enumerate(list_IDs_temp):
        # Store sample
        img = cv.imread(ID)
        img_resized = cv.resize(img, self.dim[::-1])          
        X[i,] = img_resized

        # Store class
        y[i] = list_labels_temp[i]

    return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
# generated data train test split
train_generator = DataGenerator(list_IDs= images_paths_train, labels= images_labels_train)
validation_generator = DataGenerator(list_IDs= images_paths_validation, labels= images_labels_validation)
test_generator = DataGenerator(list_IDs= images_paths_test, labels= images_labels_test)

In [ ]:
CLASSES_labels

In [ ]:
# make sure everything works fine
for images, labels in train_generator:
  print(images.shape)
  print(labels.shape)
  for index in range(len(labels)):
    cv2_imshow(images[index])
    print(labels[index])
  break

# **Model**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Dropout, Dense, Conv2D, MaxPooling2D, Flatten, Input
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
model = tf.keras.Sequential()
model.add(Input(shape=(64,128, 3)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
acc_checkpoint = ModelCheckpoint('best_accuracy.h5', monitor="val_accuracy")

In [ ]:
model.fit(x = train_generator,
          validation_data = validation_generator,
          epochs=15,
          callbacks=[acc_checkpoint])